In [1]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
import statsmodels.api
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.model_selection import cross_val_score,KFold,GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import warnings
warnings.filterwarnings('ignore')

In [2]:
df =pd.read_csv("music30s_trainin.csv")
features=df.columns

In [3]:
df.drop(['length',"Unnamed: 0"], axis = 1)

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,0.432846,0.086019,0.142929,0.004758,3089.637185,1.311379e+06,2973.703815,271609.298651,6574.771380,4.659712e+06,...,-4.922164,106.523758,-5.005428,74.781494,1.383756,75.593880,-2.103029,82.962410,-2.102471,56.383205
1,0.398761,0.090296,0.092221,0.001716,2170.544505,2.470449e+05,2054.465986,111598.068567,4306.240937,1.063089e+06,...,5.730653,62.366760,-7.947048,57.781807,5.203198,71.078148,-1.486348,55.774078,0.463148,60.979748
2,0.375874,0.081557,0.099977,0.001147,2321.871533,5.433791e+05,2545.325000,155899.829366,4580.418732,2.651598e+06,...,-0.442143,51.811581,-3.584000,51.066288,-5.018315,47.032211,-2.522085,35.879490,-4.752133,33.392231
3,0.424385,0.085162,0.175975,0.003569,1886.954809,2.533910e+05,1913.258610,123441.419116,3828.288761,1.247085e+06,...,0.975607,48.149387,-6.554968,35.716187,-0.689079,46.263088,-2.127222,47.571030,-1.508367,50.199821
4,0.456134,0.075901,0.097609,0.000364,2269.128009,1.357942e+05,1928.953538,88450.196573,4186.459372,7.265238e+05,...,6.971109,40.836742,-12.668804,40.830875,-0.556885,31.079792,-11.754225,37.405830,-1.540814,30.325340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.469879,0.086839,0.144261,0.008266,2868.233682,1.353624e+06,2676.125867,225551.708478,5796.886499,3.569678e+06,...,1.998499,69.473038,-5.572653,90.371826,7.771932,123.477547,-1.518996,96.361115,5.007198,67.499344
896,0.290960,0.091076,0.089215,0.003113,1109.352784,1.386997e+05,1463.217702,37692.267055,2243.341758,8.382376e+05,...,-0.187531,100.648956,-2.823469,87.407570,-2.675381,77.598251,0.740614,76.061050,-0.530982,67.630028
897,0.454745,0.086461,0.223571,0.013054,3123.151410,9.471130e+05,2970.246872,221789.180783,6616.305648,3.583799e+06,...,-1.443429,46.262592,-1.865332,50.931961,-0.019046,65.033195,-0.727255,73.956261,-0.986098,66.340393
898,0.437934,0.094721,0.252890,0.013661,1821.519761,7.452975e+05,2522.060569,309565.051070,4003.377184,4.796941e+06,...,-0.989756,49.640724,-5.466094,51.848621,-1.297010,42.504318,-4.247594,39.925827,-5.021968,58.707275


In [4]:
df_tar =pd.read_csv("music30s_trainlabel.csv")
df_tar.drop(['Unnamed: 0'],axis=1)
le = LabelEncoder()
df_tar = le.fit_transform(df_tar['label'])

In [5]:
scaler = MinMaxScaler()
 
df_scaled = scaler.fit_transform(df.to_numpy())
df_scaled = pd.DataFrame(df_scaled, columns=features)

In [6]:
pipe_lr = Pipeline([('clf', LogisticRegression(multi_class='ovr',penalty='l2',solver='newton-cg',C=11))])

pipe_rf = Pipeline([('clf', RandomForestClassifier(max_depth= 11, min_samples_leaf= 4, n_estimators= 1000,criterion= 'entropy'))])

pipe_svm = Pipeline([('clf', svm.SVC(degree=3, gamma= 'scale', kernel= 'poly'))])

pipe_nb = Pipeline([('clf', GaussianNB(var_smoothing= 1e-08))])

pipe_knn = Pipeline([('clf', KNeighborsClassifier(n_neighbors=5,metric='seuclidean'))])
pipelines=[pipe_lr,pipe_rf,pipe_svm,pipe_nb,pipe_knn]

In [7]:
for pipe in pipelines:
    best_score=0
    best_component=0
    for i in range(2,57):
        df_trans_chi= SelectKBest(chi2, k=i).fit_transform(df_scaled,df_tar)
        stratifiedkf=StratifiedKFold(n_splits=5)
        score=cross_val_score(pipe,df_trans_chi,df_tar,cv=stratifiedkf).mean()
        if score>best_score:
            best_score=score
            best_component=i
    print("best score for ",pipe,"with components",best_component,"accuracy is ",best_score)
    

best score for  Pipeline(steps=[('clf',
                 LogisticRegression(C=11, multi_class='ovr',
                                    solver='newton-cg'))]) with components 56 accuracy is  0.7066666666666667
best score for  Pipeline(steps=[('clf',
                 RandomForestClassifier(criterion='entropy', max_depth=11,
                                        min_samples_leaf=4,
                                        n_estimators=1000))]) with components 56 accuracy is  0.7677777777777777
best score for  Pipeline(steps=[('clf', SVC(kernel='poly'))]) with components 52 accuracy is  0.7422222222222222
best score for  Pipeline(steps=[('clf', GaussianNB(var_smoothing=1e-08))]) with components 56 accuracy is  0.5444444444444445
best score for  Pipeline(steps=[('clf', KNeighborsClassifier(metric='seuclidean'))]) with components 52 accuracy is  0.6811111111111112
